In [1]:
'''
# if working on laptop on local docker:
docker run -p 4242:8888 -v ~/dl_cas/:/notebooks -p 6006:6006 -it oduerr/tf_docker:tf1_py3
'''

'\n# if working on laptop on local docker:\ndocker run -p 4242:8888 -v ~/dl_cas/:/notebooks -p 6006:6006 -it oduerr/tf_docker:tf1_py3\n'

## Imports

In [2]:
# python module imports needed in customized functions:
import numpy as np
import tensorflow as tf
import sys

# import customized functions collected in my_fct_for_fc_ladder.py
from my_fct_for_fc_ladder_Lilach_vb import convertToOneHot
from my_fct_for_fc_ladder_Lilach_vb import my_norm
from my_fct_for_fc_ladder_Lilach_vb import my_fc_bn
from my_fct_for_fc_ladder_Lilach_vb import my_noise
from my_fct_for_fc_ladder_Lilach_vb import my_comb_vanilla
from my_fct_for_fc_ladder_Lilach_vb import encoder_prop
from my_fct_for_fc_ladder_Lilach_vb import decoder_prop
#from my_fct_for_fc_ladder_Lilach_vb import init_weights
from my_fct_for_fc_ladder_Lilach_vb import init_weights_v2
#from my_fct_for_fc_ladder_Lilach import init_weights_combiner_vanilla
#from my_fct_for_fc_ladder_Lilach import reconst_loss

#from my_fct_for_fc_ladder_Lilach_debug import decoder_prop_ver ## for debugging

In [3]:
# additional imports of python modules

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import time
import pandas as pd
#tf.set_random_seed(1)
#np.random.seed(1)
import sys
tf.__version__, sys.version_info



('1.0.0',
 sys.version_info(major=3, minor=4, micro=3, releaselevel='final', serial=0))

# Data read-in 

### Load small external MNIST data set when for working local on windows

In [4]:

# upload mnist_4000.pkl.gz which we have used in the DL course to home
# To be compatible with python3 and python2
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip

with gzip.open('mnist_4000.pkl.gz', 'rb') as f:
    if sys.version_info.major > 2:
        (X,y) = pickle.load(f, encoding='latin1')
    else:
        (X,y) = pickle.load(f)
PIXELS = len(X[0,0,0,:])

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

X = X.reshape([4000, 784])
#X = X/255 # is already normalized

print("small data before split X.shape", X.shape)
print("small data before  y.shape", y.shape) 

x_train = X[0:3000]
y_train = y[0:3000]
x_test = X[3000:4000]
y_test = y[3000:4000]


print("small data x_train.shape:", x_train.shape)
print("small data y_train.shape:",y_train.shape)
print("small data x_test.shape:",x_test.shape)
print("small data y_test.shape:",y_test.shape)

num_class= len(np.unique(y))
print("num_class:",num_class)



small data before split X.shape (4000, 784)
small data before  y.shape (4000,)
small data x_train.shape: (3000, 784)
small data y_train.shape: (3000,)
small data x_test.shape: (1000, 784)
small data y_test.shape: (1000,)
num_class: 10


### Or load full MNIST dataset directly from internet

In [5]:
'''
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# if images are not flatten (like in mnist) we need first to flatten them
# now flatten images for fc ladder

x_train = x_train.reshape(-1,784)
x_test = x_test.reshape(-1,784)

print("large data x_train.shape:", x_train.shape)
print("large data y_train.shape:",y_train.shape)
print("large data x_test.shape:",x_test.shape)
print("large data x_test.shape:",y_test.shape)
'''

'\nfrom keras.datasets import mnist\n\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\n# if images are not flatten (like in mnist) we need first to flatten them\n# now flatten images for fc ladder\n\nx_train = x_train.reshape(-1,784)\nx_test = x_test.reshape(-1,784)\n\nprint("large data x_train.shape:", x_train.shape)\nprint("large data y_train.shape:",y_train.shape)\nprint("large data x_test.shape:",x_test.shape)\nprint("large data x_test.shape:",y_test.shape)\n'

In [6]:
x_train[:200].shape

(200, 784)

In [7]:
np.mean(x_train[:,200])

-0.0042952602

In [8]:
np.std(x_train[:,200])

0.83271211

# Lets construct a fc NN (784->500->50->10) without noise and unsupervised task to get a benchmark for the loss and accuracy

In [9]:
# reset the default graph

tf.reset_default_graph()

In [10]:
# define placeholder which we need later to feed in our data:
# be sure that input data is normalized
x = tf.placeholder(tf.float32, shape=[None, 784], name='x_data')
y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_data')

In [11]:
# our benchmark model hast 3 hidden layers
# x:h0:784 -> h1:500 -> h2:50 -> h3:10 (softmax)

# initialization
############################################
# we use exactly the sampe architecture and initialization procedure like in ladder
    
# initialization of learnable variables (see custum fct init_weights)
# predefine intialization of variables which are updated during trianing:
# we use no biases since we do batchnorm (following Martin Görner)
# Bs are used as offset in batchnorm (no scales in bn since we use ReLu)

my_sd = 0.1  # sd for initialization of W with N(0,sd)
fan = 10     # for dividing the 1-initialized B

W1 = tf.Variable(tf.random_uniform(shape=[784, 500],minval=-0.05, maxval=0.05))  
Blt1 = tf.Variable(tf.zeros([500]))
B1 = tf.Variable(tf.zeros([500]))
W2 = tf.Variable(tf.random_uniform(shape=[500, 50],minval=-0.05, maxval=0.05))
Blt2 = tf.Variable(tf.zeros([50]))
B2 = tf.Variable(tf.zeros([50]))
W3 = tf.Variable(tf.random_uniform(shape=[50, 10],minval=-0.05, maxval=0.05))
Blt3 = tf.Variable(tf.zeros([10]))
B3 = tf.Variable(tf.zeros([10]))
#gamma = tf.Variable(tf.random_uniform(shape=[10],minval=-0.05, maxval=0.05)) 


In [12]:
# define the model architecture as encoder in ladder: 
# TODO maby skip my_fc_bn?

x = my_norm(x, "initial_z_trafo")

hn_lt_1 = tf.matmul(x, W1) + Blt1 
hn_bn_1 = my_fc_bn(Ylogits=hn_lt_1, offset=B1, scope="bn")
hn_nlt_1 = tf.nn.relu(hn_bn_1)
 
hn_lt_2 = tf.matmul(hn_nlt_1, W2) + Blt2
hn_bn_2 = my_fc_bn(Ylogits=hn_lt_2, offset=B2, scope="bn")
hn_nlt_2 = tf.nn.relu(hn_bn_2)

hn_lt_3 = tf.matmul(hn_nlt_2, W3) + Blt3
hn_bn_3 = hn_lt_3 #my_fc_bn(Ylogits=hn_lt_3, offset=B3, scope="bn")
out = tf.nn.softmax(hn_bn_3)  # TODO : IS THIS ERROR PRONE? 


In [13]:
########################################################################
# supervised loss
############################################################################

# cross-entropy loss function (= -sum(Y_i * log(Yi)) ), 
# TensorFlow provides the softmax_cross_entropy_with_logits function to avoid numerical stability
# problems with log(0) which is NaN

with tf.name_scope("loss_supervised"):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=hn_bn_3, labels=y_true)
    # loss from supervised learning:
    loss_supervised = tf.reduce_mean(cross_entropy) 

In [14]:
with tf.name_scope("initialize"):
    init_op = tf.global_variables_initializer() 

In [15]:
# train by using  SGD Optimizer
with tf.name_scope("train_step"):
    train_op = tf.train.GradientDescentOptimizer(0.01).minimize(loss_supervised)

In [16]:
#We want to visualize the development of the following variables in tensorboard:
for v in tf.trainable_variables():
    print("Adding ", v.name)
    tf.summary.histogram(v.name, v)

Adding  Variable:0
INFO:tensorflow:Summary name Variable:0 is illegal; using Variable_0 instead.
Adding  Variable_1:0
INFO:tensorflow:Summary name Variable_1:0 is illegal; using Variable_1_0 instead.
Adding  Variable_2:0
INFO:tensorflow:Summary name Variable_2:0 is illegal; using Variable_2_0 instead.
Adding  Variable_3:0
INFO:tensorflow:Summary name Variable_3:0 is illegal; using Variable_3_0 instead.
Adding  Variable_4:0
INFO:tensorflow:Summary name Variable_4:0 is illegal; using Variable_4_0 instead.
Adding  Variable_5:0
INFO:tensorflow:Summary name Variable_5:0 is illegal; using Variable_5_0 instead.
Adding  Variable_6:0
INFO:tensorflow:Summary name Variable_6:0 is illegal; using Variable_6_0 instead.
Adding  Variable_7:0
INFO:tensorflow:Summary name Variable_7:0 is illegal; using Variable_7_0 instead.
Adding  Variable_8:0
INFO:tensorflow:Summary name Variable_8:0 is illegal; using Variable_8_0 instead.


In [17]:
# We want to visualize the development of the loss in tensorboard

tf.summary.scalar("loss_supervised", loss_supervised)

<tf.Tensor 'loss_supervised_1:0' shape=() dtype=string>

In [18]:
! rm -rf /tmp/ladder

In [19]:
! mkdir /tmp/ladder

In [20]:
! ls /tmp/ladder

In [21]:
# collect all summaries for tensorboard and define the directory for saved summary files 

merged_summary_op = tf.summary.merge_all()
summary_writer = tf.summary.FileWriter("/tmp/ladder", tf.get_default_graph())

In [22]:
sess = tf.Session() 
sess.run(init_op)

In [24]:
# check the shape of the feeds:
#x = tf.placeholder(tf.float32, shape=[None, 784], name='x_data')
#y_true = tf.placeholder(tf.float32, shape=[None, 10], name='y_data')
print("x_train.shape:", x_train.shape)  
print("convertToOneHot(y_train, 10).shape:", convertToOneHot(y_train, 10).shape)  #

x_train.shape: (3000, 784)
convertToOneHot(y_train, 10).shape: (3000, 10)


In [25]:
vals = []
for i in range(5000):
    idx = np.random.permutation(len(x_train))[0:128] #Easy minibatch of size 128
    loss_, _, res_ = sess.run((loss_supervised, train_op, out), 
                              feed_dict={x:x_train[idx], y_true:convertToOneHot(y_train[idx], 10)})
    if (i % 100 == 0):#50
        acc = np.average(np.argmax(res_, axis = 1) == y_train[idx])
        # Get the results for the validation results 
        loss_v, res_val, summary_ = sess.run([loss_supervised, out, merged_summary_op], 
                                                          feed_dict={x:x_test, 
                                                                     y_true:convertToOneHot(y_test, 10)})
        summary_writer.add_summary(summary_, i)
        acc_v = np.average(np.argmax(res_val, axis = 1) == y_test)
        vals.append([loss_, acc, loss_v, acc_v])
        print("{} Training: loss {} acc {} Validation: loss {} acc {}".format(i, loss_, acc, loss_v, acc_v))
 

0 Training: loss 2.319735050201416 acc 0.046875 Validation: loss 2.3231749534606934 acc 0.082
100 Training: loss 1.7501676082611084 acc 0.6875 Validation: loss 1.7814929485321045 acc 0.65
200 Training: loss 1.2755136489868164 acc 0.703125 Validation: loss 1.2843124866485596 acc 0.71
300 Training: loss 0.8503386974334717 acc 0.859375 Validation: loss 0.9666263461112976 acc 0.8
400 Training: loss 0.6365038752555847 acc 0.90625 Validation: loss 0.7514290809631348 acc 0.857
500 Training: loss 0.459953635931015 acc 0.9375 Validation: loss 0.6060840487480164 acc 0.882
600 Training: loss 0.41558706760406494 acc 0.953125 Validation: loss 0.5162840485572815 acc 0.892
700 Training: loss 0.35481980443000793 acc 0.9296875 Validation: loss 0.4534491300582886 acc 0.902
800 Training: loss 0.27904239296913147 acc 0.96875 Validation: loss 0.410434365272522 acc 0.906
900 Training: loss 0.2426457405090332 acc 0.953125 Validation: loss 0.38147205114364624 acc 0.906
1000 Training: loss 0.15311047434806824 

In [ ]:
# compare to directory in tf.summary.FileWriter
! tensorboard --logdir /tmp/ladder/
# check docker call and go to http://srv-lab-t-697:8711